In [1]:
from keras.layers import Input, Dense
from keras.models import load_model, Sequential
import numpy as np

encoding_dim = 32
image_x, image_y = (336, 300)
image_dim = image_x * image_y

import csv
import os
from multiprocessing import Process, Lock, Manager

def run(psID, data, fnames, printlock, datalock):
    with printlock:
      print("Subprocess "+str(psID)+" starting!")
    try:
      while True:
        fname = fnames.pop()
        if fname.endswith("0.csv"):
          with printlock:
            print(fname)
          with open("../csv/" + fname, "r") as file:
            try:
              file_frames = []
              while True:
                curr = len(file_frames)
                file_frames.append(bytearray())
                #with printlock:
                #  print "Starting", curr
                #  print len(data)
                while True:
                  line = next(file)
                  if len(line) <= 1:
                    break
                  file_frames[curr].extend(int(i) for i in line.split(','))
            except StopIteration:
              with datalock:
                data.extend(file_frames)
    except IndexError:
      with printlock:
        print("Subprocess "+str(psID)+" exiting!")

import cPickle as pickle
try:
  print "Attempting to load rawdata.pickle..."
  with open("rawdata.pickle", "r") as f:
    data = pickle.load(f)
  print "Success"
except IOError:
  print "Failed, loading from ../csv..."
  with Manager() as manager:
    data = manager.list()
    fnames = manager.list(os.listdir("../csv"))
    printlock = Lock()
    datalock = Lock()

    nCores = 40
    processes = []
    for i in range(nCores):
      p = Process(target = run, args = (i,data,fnames,printlock,datalock))
      processes.append(p)

    for p in processes:
      p.start()
    for p in processes:
      p.join()

    data = list(data)

    print("Done! Pickling...")

    with open("rawdata.pickle", "wb") as p:
      pickle.dump(data, p, 2)

    print("Wrote rawdata.pickle")


Using TensorFlow backend.


Attempting to load rawdata.pickle...
Success


In [14]:
data = [[int(x) for x in ba] for ba in data if len(ba) == 100800]


import math
d = int(math.floor(0.8 * len(data)))
x_train = np.array(data[:d])
x_test = np.array(data[d:])

#from keras.datasets import mnist
#x_train = x_train.astype('float32') / 255.
#x_test = x_test.astype('float32') / 255.
#x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
#x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))


In [13]:
from keras.backend import shape
str(shape(x_train))

'Tensor("Shape_10:0", shape=(2,), dtype=int32)'

In [ ]:

try:
  print("Attempting to load model...")
  autoencoder = load_model('model.h5')
  print("Success.")
except IOError:
  print("Failed, training a model.")
  autoencoder = Sequential([
    Dense(image_dim * 2, activation = 'relu', input_dim = image_dim),
    Dense(encoding_dim, activation='relu'),
    Dense(image_dim * 2, activation = 'relu'),
    Dense(image_dim, activation='sigmoid')
  ])

  autoencoder.compile(optimizer='adam', loss='mean_squared_error')

  autoencoder.fit(x_train, x_train,
#                epochs=5,
#                batch_size=1,
#                shuffle=True,
#                validation_data=(x_test, x_test)
  )
  print "Done training. Saving model.h5..."
  autoencoder.save('model.h5')
  print "Saved."


In [ ]:

# encode and decode some digits
# note that we take them from the *test* set
#decoded_imgs = get_activations(autoencoder, -1, x_test)
decoded_imgs = autoencoder.predict(x_test)

In [ ]:

# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(image_x, image_y))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(image_x, image_y))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()


In [ ]:

#from keras import backend as K
#def get_activations(model, layer, X_batch):
#    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output,])
#    activations = get_activations([X_batch,0])
#    return activations
